<a href="https://colab.research.google.com/github/abhi-11nav/Brain_tumour_Classification/blob/main/BrainTumour_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
import tensorflow as tf 
from tensorflow import keras

from tensorflow.keras.layers import Dense, Flatten, Input, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob

# Data credits : 

In [ ]:
!nvidia-smi

Mon Jun 27 15:14:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
image_size = [255,255]

training_data_path = "/content/drive/MyDrive/data/Training/"
testing_data_path = "/content/drive/MyDrive/data/Testing/"

In [ ]:
# Data Augmentation

train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.3,horizontal_flip=True,vertical_flip=True,shear_range=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
categories = glob("/content/drive/MyDrive/data/Training/*")

In [ ]:
len(categories)

4

In [ ]:
train_data = train_datagen.flow_from_directory(training_data_path,target_size=(224,224),batch_size=16,class_mode="categorical")

test_data = test_datagen.flow_from_directory(testing_data_path, target_size=(224,224),batch_size=2,class_mode="categorical")

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


## Transfer learning technique

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
vgg19 = VGG19(input_shape= image_size+[3], weights="imagenet", include_top=False)

80150528/80134624 [==============================] - 0s 0us/step


In [ ]:
for layers in vgg19.layers:
  layers.trainable = False

In [ ]:
x = Flatten()(vgg19.output)

prediction = Dense(len(categories),activation="softmax")(x)

In [ ]:
model = Model(inputs=vgg19.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 255, 255, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 255, 255, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 255, 255, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 127, 127, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 127, 127, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 127, 127, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 63, 63, 128)       0     

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_data,validation_data=test_data,epochs=10,steps_per_epoch= len(train_data),validation_steps= len(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
  5/180 [..............................] - ETA: 11:38 - loss: 2.3978 - accuracy: 0.3000